In [1]:
import re
import random
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU

Using TensorFlow backend.
/home/zh/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 读取数据, 生成汉字列表

with open('poetry.txt') as f:
    raw_text = f.read()
lines = raw_text.split("\n")[:-1]
poem_text = [i.split(':')[1] for i in lines]
char_list = [re.findall('[\x80-\xff]{3}|[\w\W]', s) for s in poem_text]

In [3]:
# 汉字 <-> 数字 映射

all_words = []
for i in char_list:
    all_words.extend(i)
word_dataframe = pd.DataFrame(pd.Series(all_words).value_counts())
word_dataframe['id'] = list(range(1,len(word_dataframe)+1))

word_index_dict = word_dataframe['id'].to_dict()
index_dict = {}
for k in word_index_dict:
    index_dict.update({word_index_dict[k]:k})
    
len(all_words), len(word_dataframe), len(index_dict)

(217689, 4619, 4619)

In [4]:
# 生成训练数据, x 为 前两个汉字, y 为 接下来的汉字 
# 如: 明月几时有 会被整理成下面三条数据
# 明月 -> 几  月几 -> 时  几时 -> 有

seq_len = 2
dataX = []
dataY = []
for i in range(0, len(all_words) - seq_len, 1):
    seq_in = all_words[i : i + seq_len]
    seq_out = all_words[i + seq_len]
    dataX.append([word_index_dict[x] for x in seq_in])
    dataY.append(word_index_dict[seq_out])

len(dataY)

217687

In [5]:
X = np.array(dataX)
y = np_utils.to_categorical(np.array(dataY))
X.shape, y.shape

((217687, 2), (217687, 4620))

In [6]:
model = Sequential()

# Embedding 层将正整数（下标）转换为具有固定大小的向量，如[[4],[20]]->[[0.25,0.1],[0.6,-0.2]]
# Embedding 层只能作为模型的第一层
# input_dim：大或等于0的整数，字典长度
# output_dim：大于0的整数，代表全连接嵌入的维度
model.add(Embedding(len(word_dataframe), 512))

# LSTM
model.add(LSTM(512))

# Dropout 防止过拟合
model.add(Dropout(0.5))

# output 为 y 的维度
model.add(Dense(y.shape[1]))

model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 512)         2364928   
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4620)              2370060   
_________________________________________________________________
activation_1 (Activation)    (None, 4620)              0         
Total params: 6,834,188
Trainable params: 6,834,188
Non-trainable params: 0
_________________________________________________________________


In [7]:
# 训练

model.fit(X, y, batch_size=64, epochs=40)

Epoch 1/40
217687/217687 [==============================] - 47s - loss: 6.5194    
Epoch 2/40
217687/217687 [==============================] - 43s - loss: 6.0894    
Epoch 3/40
217687/217687 [==============================] - 44s - loss: 5.7482    
Epoch 4/40
217687/217687 [==============================] - 44s - loss: 5.4671    
Epoch 5/40
217687/217687 [==============================] - 44s - loss: 5.2141    
Epoch 6/40
217687/217687 [==============================] - 43s - loss: 4.9809    
Epoch 7/40
217687/217687 [==============================] - 43s - loss: 4.7727    
Epoch 8/40
217687/217687 [==============================] - 43s - loss: 4.5822    
Epoch 9/40
217687/217687 [==============================] - 43s - loss: 4.4151    
Epoch 10/40
217687/217687 [==============================] - 43s - loss: 4.2630    
Epoch 11/40
217687/217687 [==============================] - 44s - loss: 4.1322    
Epoch 12/40
217687/217687 [==============================] - 43s - loss: 4.0099    
E

In [8]:
def get_predict_array(seed_text):
    chars = re.findall('[\x80-\xff]{3}|[\w\W]', seed_text)
    x = np.array([word_index_dict[k] for k in chars])
    proba = model.predict(x, verbose=0)
    return proba

get_predict_array("明月")

# 可以看到预测出来的结果是两个列表, 下一个字是第二个列表

array([[9.0341976e-15, 2.1907965e-02, 1.1279847e-01, ..., 1.7754996e-10,
        1.7678926e-14, 1.0797626e-09],
       [3.9345502e-12, 1.9920176e-01, 2.0544907e-02, ..., 1.1154208e-08,
        1.1879822e-12, 1.6002797e-08]], dtype=float32)

In [9]:
def gen_poetry(model, seed_text, rows=4, cols=5):
    '''
    生成诗词的函数
    输入: 两个汉字, 行数, 每行的字数 (默认为五言绝句)
    '''
    total_cols = cols + 1  # 加上标点符号
    chars = re.findall('[\x80-\xff]{3}|[\w\W]', seed_text)
    if len(chars) != seq_len: # seq_len = 2
        return ""
    arr = [word_index_dict[k] for k in chars]
    for i in range(seq_len, rows * total_cols):
        if (i+1) % total_cols == 0:  # 逗号或句号
            if (i+1) / total_cols == 2 or (i+1) / total_cols == 4:  # 句号的情况
                arr.append(2)  # 句号在字典中的映射为 2
            else:
                arr.append(1)  # 逗号在字典中的映射为 1
        else:
            proba = model.predict(np.array(arr[-seq_len:]), verbose=0)
            predicted = np.argsort(proba[1])[-5:]
            index = random.randint(0,len(predicted)-1)  # 在前五个可能结果里随机取, 避免每次都是同样的结果
            new_char = predicted[index]
            while new_char == 1 or new_char == 2:  # 如果是逗号或句号, 应该重新换一个
                index = random.randint(0,len(predicted)-1)
                new_char = predicted[index]
            arr.append(new_char)
    poem = [index_dict[i] for i in arr]
    return "".join(poem)

In [10]:
print(gen_poetry(model, '明月'))
print(gen_poetry(model, '悠然', rows=4, cols=7))
print(gen_poetry(model, '长河', rows=4, cols=7))

明月明月上，今已见天涯。长信多异迹，不见月明德。
悠然长城头一朝，不见山下流水曲。不得意切玉帛事，今在何处不可见。
长河边草生得不，今在长安得不可。玉树乌号边草绿，万国恩光辉照海。


In [11]:
model.save(filepath='lstm_poetry.hdf5')

In [12]:
# 试下 GRU

gru = Sequential()
gru.add(Embedding(len(word_dataframe), 512))
gru.add(GRU(512))
# gru.add(Dropout(0.5))
gru.add(Dense(y.shape[1]))
gru.add(Activation('softmax'))
gru.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
gru.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 512)         2364928   
_________________________________________________________________
gru_1 (GRU)                  (None, 512)               1574400   
_________________________________________________________________
dense_2 (Dense)              (None, 4620)              2370060   
_________________________________________________________________
activation_2 (Activation)    (None, 4620)              0         
Total params: 6,309,388
Trainable params: 6,309,388
Non-trainable params: 0
_________________________________________________________________


In [14]:
gru.fit(X, y, batch_size=64, epochs=40)

Epoch 1/40
217687/217687 [==============================] - 39s - loss: 6.3603    
Epoch 2/40
217687/217687 [==============================] - 39s - loss: 5.6961    
Epoch 3/40
217687/217687 [==============================] - 39s - loss: 5.1456    
Epoch 4/40
217687/217687 [==============================] - 39s - loss: 4.6007    
Epoch 5/40
217687/217687 [==============================] - 39s - loss: 4.1190    
Epoch 6/40
217687/217687 [==============================] - 39s - loss: 3.7241    
Epoch 7/40
217687/217687 [==============================] - 39s - loss: 3.4112    
Epoch 8/40
217687/217687 [==============================] - 39s - loss: 3.1593    
Epoch 9/40
217687/217687 [==============================] - 39s - loss: 2.9692    
Epoch 10/40
217687/217687 [==============================] - 39s - loss: 2.8208    
Epoch 11/40
217687/217687 [==============================] - 39s - loss: 2.7050    
Epoch 12/40
217687/217687 [==============================] - 39s - loss: 2.6151    
E

In [15]:
print(gen_poetry(gru, '明月'))
print(gen_poetry(gru, '悠然', rows=4, cols=7))
print(gen_poetry(gru, '长河', rows=4, cols=7))

明月落叶下，玉关塞垣月。风声向北游，寒催急春芳。
悠然知我德惭不，万行犹在昭享诚。玉门前歌欲亲哀，天白山路遥行路。
长河边思君不行，万重轩行未安得。玉关塞天白日月，玉门前鸟飞花似。


In [16]:
gru.save('gru_poetry.hdf5')